In [1]:
import os
import json
import fasttext
from tqdm import tqdm
import json
# from transformers import AutoTokenizer

In [2]:
model = fasttext.load_model("/data/hyxu/codes/lowMT_compute/model/lid.176.ftz")
model, model.predict("sadnasjhfn iab asina winawfh siefnwvjawoi saf qwhoiasoidfw9 f as jfwoias;lk ")

(<fasttext.FastText._FastText at 0x7fe5d4b36470>,
 (('__label__nl',), array([0.38591853])))

In [4]:

data_path = "/data/hyxu/codes/lowMT_compute/data/public_data/train/mono"
def _read(p):
    with open(p) as f:
        data = f.readlines()
    return [s.strip("\n") for s in data]
nl_data = _read(os.path.join(data_path, "train.nl"))
zh_data = _read(os.path.join(data_path, "train.zh"))

In [5]:
lang1, lang2 = "nl", "zh"
data_socre = {lang1: [], lang2: []}
r = 0.4
nl_data_result =[]
for s in tqdm(nl_data):
    result = model.predict(s)
    if result[0][0] == f"__label__{lang1}" and result[1] > r:
        nl_data_result.append((s, result[1].item()))
zh_data_result =[]
for s in tqdm(zh_data):
    result = model.predict(s)
    if result[0][0] == f"__label__{lang2}" and result[1] > r:
        zh_data_result.append((s, result[1].item()))
len(nl_data_result), len(zh_data_result)

100%|██████████| 8000000/8000000 [02:11<00:00, 60879.49it/s]


In [16]:
with open(os.path.join(data_path, "clean.train.nl"), "w") as f:
    json.dump(nl_data_result, f, indent=4)
with open(os.path.join(data_path, "clean.train.zh"), "w") as f:
    json.dump(zh_data_result, f, indent=4)

### 挖掘双语中潜藏的双语语料

只挖掘中文

In [8]:
zh_data = _read(os.path.join("/data/hyxu/codes/lowMT_compute/data/public_data/train/pair/train.zh"))
from tqdm import tqdm
r = 0.5
results_data = []
for zh_s in tqdm(zh_data):
    zh_r = model.predict(zh_s)
    if zh_r[0][0] != "__label__zh" or zh_r[1] < r:
        results_data.append(zh_s)        
len(results_data)

100%|██████████| 200000/200000 [00:02<00:00, 76283.74it/s]


38306

In [20]:
import re
pat = "[\u4e00-\u9fa5\0-9\,\。]"
nl_text = [re.sub(pat, " ", s) for s in results_data]
nl_text = [re.sub("[\s]+", " ", s) for s in nl_text]
pat = "[A-Za-z0-9\,\。]"
zh_text = [re.sub(pat, " ", s) for s in results_data]
zh_text = [re.sub("[\s]+", "", s) for s in zh_text]
list(zip(nl_text, zh_text))

[(' ，tymek ', '哦，'),
 ('Elena ', '.'),
 (' ', '刚到不久'),
 (' ? Well you can t say you didn t have a part in that?',
  "但是你不能说你没有一点责任?''?"),
 (' ', '再见小美人一会儿见宝贝还有'),
 ('maybe I can get out of my cell {\\ cH }And maybe I can get into the guards room {\\ cH }I ll still be in that room {\\ cH }I ll be sealed in ',
  ".也许我能进到警卫室{\\}'.我仍会在那个房间{\\}'.已经被封住了你叫我怎么出去{\\}'."),
 (' ', '-他还活着'),
 (' ', '真的对不起'),
 (' ', '-不会丢失的钥匙圈'),
 (' You were so quiet and pasty and ', '你很安静脸色苍白而且......特别聪明帅气...'),
 (' ?', '冒险就为了铅?'),
 ('CSenarak:', ':'),
 (' ', '学着点仙女棒'),
 (' ', '南美财富及希望之地'),
 (' Well you certainly gave Mr Rhubarb a run for his money ',
  '你完全抢了小丑先生的风头..'),
 (' ', '-玩得开心点'),
 (' ， ， ， ', '-哦，我的天，你在流血，宝贝'),
 (' ， ', '-不，也不全然吧'),
 (' Then I ll resign from Scorpion effective immediately ', "好吧那我立刻退出天蝎'."),
 (' Natania put the gun down ', '娜塔尼亚放下枪.'),
 (' Then buckle up you re in for a cranky night According to the codicil of the Relationship Agreement which you insisted upon ',
  "那系好安全带今晚会天摇地动的'.你一向坚

In [22]:
data_len = [(s, t) for s, t in (zip(nl_text, zh_text)) if len(s) > 4 and len(t) >4 ]
len(data_len), data_len

(10232,
 [(' ? Well you can t say you didn t have a part in that?',
   "但是你不能说你没有一点责任?''?"),
  ('maybe I can get out of my cell {\\ cH }And maybe I can get into the guards room {\\ cH }I ll still be in that room {\\ cH }I ll be sealed in ',
   ".也许我能进到警卫室{\\}'.我仍会在那个房间{\\}'.已经被封住了你叫我怎么出去{\\}'."),
  (' You were so quiet and pasty and ', '你很安静脸色苍白而且......特别聪明帅气...'),
  (' Well you certainly gave Mr Rhubarb a run for his money ',
   '你完全抢了小丑先生的风头..'),
  (' ， ， ， ', '-哦，我的天，你在流血，宝贝'),
  (' Then I ll resign from Scorpion effective immediately ', "好吧那我立刻退出天蝎'."),
  (' Natania put the gun down ', '娜塔尼亚放下枪.'),
  (' Then buckle up you re in for a cranky night According to the codicil of the Relationship Agreement which you insisted upon ',
   "那系好安全带今晚会天摇地动的'.你一向坚持贯彻的恋爱条例的附加条约里写过"),
  (' If this club goes to the wall It ll be a disaster ', "吉姆这支队伍垮了损失可惨重了'."),
  (' Good dogs don t bite ', "好狗不咬人'...."),
  (' At least she s getting back out there ', "至少她现在重新开始约会了'."),
  (' ・ James Bovie is my fa

In [25]:
data_p = []
for s, t in data_len:
    nl_r = model.predict(s)
    zh_r = model.predict(t)
    if (nl_r[0][0] != "__label__nl" and nl_r[0][0] != "__label__en") or nl_r[1] < 0.1:
        continue
    if zh_r[0][0] != "__label__zh" or zh_r[1] < 0.1:
        continue
    data_p.append((s, t))
len(data_p), data_p

(5270,
 [(' ? Well you can t say you didn t have a part in that?',
   "但是你不能说你没有一点责任?''?"),
  (' You were so quiet and pasty and ', '你很安静脸色苍白而且......特别聪明帅气...'),
  (' Well you certainly gave Mr Rhubarb a run for his money ',
   '你完全抢了小丑先生的风头..'),
  (' Then I ll resign from Scorpion effective immediately ', "好吧那我立刻退出天蝎'."),
  (' Natania put the gun down ', '娜塔尼亚放下枪.'),
  (' Then buckle up you re in for a cranky night According to the codicil of the Relationship Agreement which you insisted upon ',
   "那系好安全带今晚会天摇地动的'.你一向坚持贯彻的恋爱条例的附加条约里写过"),
  (' Good dogs don t bite ', "好狗不咬人'...."),
  (' At least she s getting back out there ', "至少她现在重新开始约会了'."),
  (' ・ James Bovie is my father ', '詹姆斯・博维是我的父亲.'),
  (' We were going to buy some cheap cattle ', '是想买一些便宜的牛.'),
  (' She asked what it was worth ', '她问值多少钱我说镑你说镑.'),
  (' Uh I took pictures made some recordings to document my time there ',
   '我拍了照片录了录音来记录在那里的时光.'),
  (' Why would agents shoot Sameen in the head only to try to patch her up?'

In [26]:
with open("/data/hyxu/codes/lowMT_compute/data/public_data/train/pair/mine.nl-zh.nl", "w") as f:
    f.write("\n".join([s[0] for s in data_p]))
with open("/data/hyxu/codes/lowMT_compute/data/public_data/train/pair/mine.nl-zh.zh", "w") as f:
    f.write("\n".join([s[1] for s in data_p]))

## 写比赛结果

In [5]:
import os
import json
import torch
from tqdm import tqdm
from transformers import MBartForConditionalGeneration, AutoTokenizer
from datasets import load_from_disk, DatasetDict, Dataset
import  xml.dom.minidom
from eval import generate, evaluate_fn

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

data_path = "/data/hyxu/codes/lowMT_compute/data/public_data/train/mono"

In [17]:
path = "/data/hyxu/lowMT_compute/data/public_data/test"
def _read_xml(p):
    dom = xml.dom.minidom.parse(p)
    root = dom.documentElement
    seg_list = root.getElementsByTagName("seg")
    data = [s.firstChild.data for s in seg_list]
    print(len(data))
    return [s.rstrip("\n") for s in data]
data = _read_xml(os.path.join(path, "test.nl-zh.nl.xml"))
data

1000


['Bij een bezoek aan de plaatselijke school waren ze geschokt door de chaotische scenes die vaak in de klas voorkwamen.',
 'Ik zal in de volgende wedstrijd zeker meer aandacht besteden aan het passeren en de offensieve verandering.',
 'Denemarken, Zweden en Finland zijn de eerste landen ter wereld die warmtepomptechnologie gebruiken om stedelijk afvalwater te behandelen.',
 'De actualiteit van deze twee processen moet worden gegarandeerd.',
 'Ik kan niet anders zeggen dan dat ik heel trots ben.',
 'Met onze ruim 400 winkels zitten we echt overal in Nederland.',
 'De Philips Airfryer is de enige Airfryer met Fat Removal-technologie',
 'Ook zegt het college dat men wil streven naar minder inhuur.',
 'De beelden worden vervolgens beoordeeld met kunstmatige intelligentie.',
 'Bij opsporingsberichtgeving dient het OM strikt de Aanwijzing Opsporingsberichtgeving na te leven.',
 'Vandaag nodigen we 66e afgestudeerden van Tsinghua Universiteit uit om het verhaal van hun graduatiedistributie te

In [3]:
model_path  = "/data/hyxu/codes/lowMT_compute/model/ft-mine/chpk-166/model"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = model.cuda()
model

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [18]:
predictions = generate(model, tokenizer, "nl_XX", "zh_CN", data, 32, 3, 128)

100%|██████████| 32/32 [00:27<00:00,  1.18it/s]


In [19]:
dom = xml.dom.minidom.parse(os.path.join(path, "test.nl-zh.zh.xml"))
root = dom.documentElement
seg_list = root.getElementsByTagName("seg")
for s, p in zip(seg_list, predictions):
    s.firstChild.data = p
with open(os.path.join(path, "test.nl-zh.zh5.xml"), "w") as f:
    dom.writexml(f)

## 用tokenizer 清洗数据

In [6]:
model_path  = "/data/hyxu/codes/lowMT_compute/model/ft-clean-smo/chpk-8000/model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer

MBartTokenizerFast(name_or_path='/data/hyxu/codes/lowMT_compute/model/ft-clean-smo/chpk-8000/model', vocab_size=250027, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False), 'additional_special_tokens': ['ar_AR', 'cs_CZ', 'de_DE', 'en_XX', 'es_XX', 'et_EE', 'fi_FI', 'fr_XX', 'gu_IN', 'hi_IN', 'it_IT', 'ja_XX', 'kk_KZ', 'ko_KR', 'lt_LT', 'lv_LV', 'my_MM', 'ne_NP', 'nl_XX', 'ro_RO', 'ru_RU', 'si_LK', 'tr_TR', 'vi_VN', 'zh_CN']}, clean_up_tokenization_spaces=True)

In [ ]:
with open(os.path.join(data_path, "clean.train.nl")) as f:
    nl_data_result = json.load(f)
with open(os.path.join(data_path, "clean.train.zh")) as f:
    zh_data_result = json.load(f)
len(nl_data_result), len(zh_data_result)

In [30]:
r = 15              # 拼接阈值
class CleanData(object):
    r =15
    clean_data = []
    noise_data = []
    def add(self, s):
        l = len(s)
        if l > r:
            self.clean_data.append(s)
        else:
            if len(self.noise_data) < 1:
                self.noise_data.append(s)
            else:
                new_string = self.noise_data.pop()
                new_string += s
                self.add(new_string)
nl_clean_data = CleanData()        
zh_clean_data = CleanData()        

In [31]:
tokenizer.tokenize("你是个大事必")

['▁你', '是个', '大事', '必']

In [32]:
from tqdm import  tqdm
for s in tqdm(nl_data_result):
    tokenized_text = tokenizer.tokenize(s[0])
    nl_clean_data.add(tokenized_text)

  1%|          | 14289/1961790 [00:22<08:24, 3858.20it/s]

In [ ]:
for s in tqdm(zh_data_result):
    tokenized_text = tokenizer.tokenize(s[0])
    zh_clean_data.add(tokenized_text)

In [ ]:
len(nl_clean_data)

## 测试验证机分数

In [6]:
model_path  = "/data/hyxu/codes/lowMT_compute/model/pre-ft-clean-smo-zh-nl/chpk-20270/model"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = model.cuda()

In [6]:
data = load_from_disk("/data/hyxu/codes/lowMT_compute/data/public_data/dev_set")

In [13]:
metrics = ["bleu",]
s, t= "nl_XX", "zh_CN"
outputs = evaluate_fn(model, tokenizer, s, t, {s:data[s], t:data[t]}, batch_size=16, num_beams=3, max_length=128,
                        metrics=metrics)
outputs[0]

100%|██████████| 63/63 [00:32<00:00,  1.93it/s]


{'bleu': 21.835933005889682}

In [10]:
outputs[1]

['打扰了 我要坐哪班火车去威海?',
 '编辑们要勇于表达自己的想法 不管他们看起来有多疯狂',
 '但有证据表明 并非每个人都会这么做',
 'Uranium是一种重型、不稳定的原子,它在核爆炸中与二到三核的微型原子相连。',
 '参观圣地亚哥神庙十分值得 它被尊为中国古代建筑杰出的范例',
 '能源危机不是方法危机,更何况不是资源危机。',
 '它利用这些信息销售产品',
 '你在思考什么是科学知识 以及如何得到它',
 '债券是一种投资行为',
 '辩护学与法律援助密不可分。',
 '我可以吃任何东西 但我觉得吃中国菜更好吃',
 '因此,法律制定者不应该违背自然法则',
 '完成硕士学位后 你可以专攻你擅长的领域',
 '消费者购买商品时 有权利得到法律上的保证',
 '申请重审需满足以下条件:',
 '太热了,我们什么时候唱?',
 '采访就被取消了 不知情的访谈取消了',
 '想法是新闻产品的灵魂',
 '汤姆是第一个到达终点线的人',
 '斯梅斯于2012年3月23日正式入选荷兰国家队',
 '你可以称它为真正的家居设计 以预算为准',
 '未来将有思维系统应用的增长预期。',
 '那些没有取得大专学历的人,有可能无法取得相应学历。',
 '红袜主帅戈尔三次致命击打对方门柱',
 '这样程序员就能轻而易举地改变程序',
 '一些中学教材和学习指南是免费的。',
 '我想买运动服,我该穿几号?',
 '亲爱的 你告诉我你昨天在餐馆吃太多了 所以你不想吃早餐 对吧?',
 '我们会把资产净额还到你户头 再过几天你就可以出院了',
 '现在金属流行在时尚圈非常盛行',
 '手机也可以是直播视频中销售商品的中介。',
 '这距离的世界纪录是荷兰人的(2.46,49)。',
 '长矛会在一次引爆中起飞 至少有30米长的引爆线',
 '我每天喝一杯咖啡 但并不多',
 '与上届世锦赛不同的是 范·加拉让球队重磅引入了前锋 维吉尔・范·狄克',
 '这份遗嘱是在两位见证人见证下所签署的',
 '记者要清楚地看到媒体的本质',
 '奥马哈必须赢才能赢得 冠军。',
 '科林・欧拉恩的"超级侠"系列是红玫瑰中的一颗璀璨之星',
 '日本人有很多海鲜 你喜欢吃鱼吗?',
 '女人跟男人一样有投票权和自立权利',
 '具体金额还在庭审中审定',
 '这些应用没有成功地改进

In [8]:
metrics = ["bleu",]
t, s = "nl_XX", "zh_CN"
outputs = evaluate_fn(model, tokenizer, s, t, {s:data[s], t:data[t]}, batch_size=20, num_beams=5, max_length=128,
                        metrics=metrics)
outputs[0], outputs[1]

  0%|          | 0/50 [00:00<?, ?it/s]/data/hyxu/codes/nmt-corrector/transformers-4.28.1/src/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (1024) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 50/50 [00:49<00:00,  1.02it/s]


({'bleu': 14.599641639797797},
 ['Wat voor trein moet ik nemen naar Wyoming?',
  'De leden van de redactie moeten hun eigen ideeën aannemen, hoe gek ze ook kunnen lijken.',
  'Maar er is bewijs dat niet iedereen zo denkt.',
  '铀 is een zwaar, onstabiel atoom, dat als er een ineenstorting plaatsvindt, in twee of meer lichte atoomkernen wordt opgesplitst, of in een of meer lichte atoomkernen.',
  'Het altaar is de moeite waard, omdat het een van de beste voorbeelden van Chinese traditionele bouw is.',
  '能源-crisis is een methodecrisis, niet een rijkdomcrisis.',
  'Het gebruikt deze informatie om klanten producten te verkopen.',
  'Je denkt na over wat wetenschap is en hoe dat is ontstaan.',
  '债券 op de obligatiemarkt.',
  'Rechtspraktijken zijn niet afhankelijk van juridische theorieën.',
  'Ik zou alles eten wat ik kan, maar ik hou meer van een lichte maaltijd.',
  'Dus de wet die de wetgevers hebben opgesteld, kan de natuur niet tegenhouden.',
  'Je gaat door met diepgaande cursussen i

## 尝试用plm做数据增强

persudo prompt

为decoder端生成时加上前缀：新闻、体育、娱乐、科技

In [34]:
import torch
from src.data_process import add_span_mask_noise

In [1]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MBartForConditionalGeneration
from transformers.models.m2m_100.configuration_m2m_100 import M2M100Config
model_path = "/data/hyxu/codes/lowMT_compute/model/pre-denoising"
# model_path = "/data/hyxu/codes/cache_dir/mbart-large-cc25"

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

class M2M(MBartForConditionalGeneration):
    def __init__(self, config: M2M100Config):
        super().__init__(config)

    def prepare_inputs_for_generation(self, decoder_input_ids, past_key_values=None, attention_mask=None, head_mask=None, decoder_head_mask=None, cross_attn_head_mask=None, use_cache=None, encoder_outputs=None, **kwargs):
        if len(decoder_input_ids[0]) == 1:
            batch_size, l = decoder_input_ids.shape
            add_tokens = torch.ones((batch_size, l+3), dtype=torch.int64, device=decoder_input_ids.device)
            add_tokens[:, 0] = decoder_input_ids[:, 0]
            add_tokens[:, 1:] = torch.tensor([6, 21606, 12])[None, :]                    # 新闻
            decoder_input_ids = add_tokens
        print(decoder_input_ids)
        return super().prepare_inputs_for_generation(decoder_input_ids, past_key_values, attention_mask, head_mask, decoder_head_mask, cross_attn_head_mask, use_cache, encoder_outputs, **kwargs)

/home/hyxu/anaconda3/envs/cor/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lang1, lang2 = "nl_XX", "zh_CN"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = MBartForConditionalGeneration.from_pretrained(model_path).cuda()

In [3]:
model

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [148]:
tokenizer.src_lang = lang2
text = "这是一个新的注意力机制，可以提高大模型的推理可扩展性。它的工作原理是将键和值投影在多个头之间共享，而不会大幅降低性能。"
encoded = tokenizer(text, return_tensors="pt")
# mask_idx = torch.randint(0, len(encoded["input_ids"][0]), (1,len(encoded["input_ids"][0]) // 4))
encoded["input_ids"] = torch.tensor(add_span_mask_noise(tokenizer.vocab_size, encoded["input_ids"][0],
                                                        noise_density=0.15, mean_noise_span_length=3)).reshape(1, -1)
encoded["attention_mask"] = encoded["input_ids"] != tokenizer.pad_token_id
# encoded["input_ids"][0, mask_idx[0]] = tokenizer.mask_token_id
# encoded["input_ids"], 
tokenizer.batch_decode(encoded["input_ids"])
# tokenizer.batch_decode(encoded["input_ids"]), mask_idx

['这是一个新的注意力机制,可以提高<mask>理可扩展性。它的工作原理是将键和值投影在多个头之间共享,而不会大幅降低性能。zh_CN']

In [150]:
for k,v in encoded.items():
    encoded[k] = v.cuda()
outputs = model.generate(**encoded, num_beams=5, decoder_start_token_id=tokenizer.lang_code_to_id[lang2])
tokenizer.batch_decode(outputs)

tensor([[250025,      6,  21606,     12],
        [250025,      6,  21606,     12],
        [250025,      6,  21606,     12],
        [250025,      6,  21606,     12],
        [250025,      6,  21606,     12]], device='cuda:0')
tensor([[250025,      6],
        [250025,   1930],
        [250025,   3893],
        [250025,  26432],
        [250025,   7153]], device='cuda:0')
tensor([[250025,      6, 100013],
        [250025,   3893,  36633],
        [250025,      6,   8513],
        [250025,   3893, 229821],
        [250025,      6,     30]], device='cuda:0')
tensor([[250025,      6, 100013,  12122],
        [250025,   3893,  36633,  39837],
        [250025,      6,     30,      6],
        [250025,      6,   8513,  12122],
        [250025,   3893, 229821,   3846]], device='cuda:0')
tensor([[250025,      6, 100013,  12122, 155026],
        [250025,      6,     30,      6, 100013],
        [250025,      6,   8513,  12122, 155026],
        [250025,   3893,  36633,  39837,     30],
        

['zh_CN 这是新的注意力机制,可以提高机理可扩展性。它的工作原理是将键和值投影在多个头之间共享,而不会大幅降低性能。</s>']

: 

In [103]:
for k,v in encoded.items():
    encoded[k] = v.cuda()
outputs = model.generate(**encoded, num_beams=5, decoder_start_token_id=tokenizer.lang_code_to_id[lang2])
tokenizer.batch_decode(outputs)

['zh_CN<s> 这是一个新的注意力机制模型的推理可扩展性。它的工作原理是将键和值投影在多个头之间共享,而不会大幅降低性能。它的工作原理是将键和值投影在多个头之间共享,而不会大幅降低性能。</s>']

## 生成nl-zh的反向翻译的数据

In [6]:
model_path  = "/data/hyxu/codes/lowMT_compute/model/ft-clean-smo/chpk-38513/model/"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = model.cuda()

In [5]:
mono_path = "/data/hyxu/codes/lowMT_compute/data/public_data/train/pair/clean.train.nl"
src_lang, tgt_lang = "nl_XX", "zh_CN"
with open(mono_path) as f:
    nl_data = f.readlines()
nl_data = [s.strip("\n") for s in nl_data]

### 再用tokenize筛一边长度，用fasttext筛一次句子

In [5]:
import fasttext
from tqdm import tqdm
import json
model = fasttext.load_model("/data/hyxu/codes/lowMT_compute/model/lid.176.ftz")

lang1, lang2 = "nl", "zh"
# data_socre = {lang1: [], lang2: []}
r = 0.7
nl_data_result =[]
for s in tqdm(nl_data):
    result = model.predict(s)
    if result[0][0] == f"__label__{lang1}" and result[1] > r:
        nl_data_result.append((s, result[1].item()))
with open("/data/hyxu/codes/lowMT_compute/data/public_data/train/mono/nl_data.json", "w") as f:
    json.dump(nl_data_result, f)
# zh_data_result =[]
# for s in tqdm(zh_data):
#     result = model.predict(s)
#     if result[0][0] == f"__label__{lang2}" and result[1] > r:
#         zh_data_result.append((s, result[1].item()))

100%|██████████| 1661930/1661930 [01:23<00:00, 19813.80it/s]


TypeError: dump() missing 1 required positional argument: 'fp'

In [7]:
from tqdm import tqdm
import json
with open("/data/hyxu/codes/lowMT_compute/data/public_data/train/mono/nl_data.json", "r") as f:
    nl_data_result = json.load(f)
nl_data = []
tokenizer.src_lang = "nl_XX"
l = 30
for s in tqdm(nl_data_result):
    input_ids = tokenizer(s[0])["input_ids"]
    if len(input_ids) > l and s[1] > 0.9:
        nl_data.append(s[0])
len(nl_data)

100%|██████████| 1631643/1631643 [07:18<00:00, 3716.94it/s]


1163539

In [8]:
nl_data[:10]

['De voorzitter werd zogenaamd aangesproken door de heer Ansing en zei dat hij wat pech had met zijn anto en dus niet eerder dan half 4 zou komen.',
 'Heb je een losliggende stoeptegel gezien, kapotte straatverlichting of andere zaken in jouw buurt? Maak er dan een foto van, meld het direct aan de gemeente, en werk zo samen met anderen aan de verbetering van jouw buurt!',
 "De tinnitus-ademtherapie is een zelfhulpprogramma met oefeningen om de psychische en fysieke oorzaken van oorsuizingen op te heffen of te verminderen. Deze uitgave is een uitgebreide en aangepaste versie van het in 2008 verschenen 'Oorsuizingen verminderen' van Maria Holl.",
 'Al met al kan ik me niet meer voorstellen waarom iemand iets anders dan kwark, tonijn, zilverrijst, kalkoen/kipfilet, visgerechten, kabeljouwfilets en brocollie zou willen. Oh ja lijnzaad zit ook in mijn dieet en dat vind ik wel lastig weg te werken nog.',
 'Geplaatst in Make-up, Reviews _ Getagged Blush, Catrice, Make-up, Reviews _ 24 Reactie

In [10]:
with open("/data/hyxu/codes/lowMT_compute/data/public_data/train/mono/bt.nl", "w") as f:
    f.write("\n".join(nl_data[:500000]))

In [7]:
# model
predictions = generate(model, tokenizer, src_lang, tgt_lang, nl_data, 24, 5, 128)
len(predictions)

  0%|          | 0/51064 [00:00<?, ?it/s]/data/hyxu/codes/nmt-corrector/transformers-4.28.1/src/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (1024) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
 11%|█         | 5547/51064 [5:10:01<42:23:56,  3.35s/it] 


KeyboardInterrupt: 

## 用zh-nl生成反向翻译数据

In [1]:
import os
import json
from transformers import AutoTokenizer, MBartForConditionalGeneration

src_lang, tgt_lang = "zh_CN", "nl_XX"

model_path = "/data/hyxu/codes/lowMT_compute/model/pre-ft-clean-smo-zh-nl/chpk-20270/model"
tokenizer =  AutoTokenizer.from_pretrained(model_path)
tokenizer

MBartTokenizerFast(name_or_path='/data/hyxu/codes/lowMT_compute/model/pre-ft-clean-smo-zh-nl/chpk-20270/model', vocab_size=250027, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False), 'additional_special_tokens': ['ar_AR', 'cs_CZ', 'de_DE', 'en_XX', 'es_XX', 'et_EE', 'fi_FI', 'fr_XX', 'gu_IN', 'hi_IN', 'it_IT', 'ja_XX', 'kk_KZ', 'ko_KR', 'lt_LT', 'lv_LV', 'my_MM', 'ne_NP', 'nl_XX', 'ro_RO', 'ru_RU', 'si_LK', 'tr_TR', 'vi_VN', 'zh_CN']}, clean_up_tokenization_spaces=True)

In [12]:
tokenizer.src_lang = src_lang
with open("/data/hyxu/codes/lowMT_compute/data/public_data/train/mono/train.zh") as f:
    data = f.readlines()
data = [s.strip("\n") for s in data if 30 > len(s) > 15]
len(data), data[:10]

(2215350,
 ['四号先生,你认为你这支花今天能送出去吗?',
  '并称这对全球人道行动将有积极的作用。',
  '他们明天就会更美味去吧,我亲爱的。',
  '计算机仿真技术应用于军事训练是当前国防领域的一项重要课题',
  '制备针状ZnO晶须影响因素的研究',
  '很好地协助总经理处理公司的重大事务与决议',
  '我不知道你们在说什么?你当然知道',
  '我们必须将该区域从冲突和战争中挽救出来。',
  '心肌缺血预适应的延迟相保护机制的研究进展',
  '中国近年来一直高速增长,您如何看中国的经济发展?'])

In [22]:
from tqdm import tqdm
results_data = []
for s in tqdm(data):
    encoded = tokenizer(s)
    if 25 > len(encoded["input_ids"]) > 10:
        results_data.append(encoded["input_ids"])
len(results_data), results_data[:10]

100%|██████████| 2215350/2215350 [06:42<00:00, 5503.79it/s]


(2145388,
 [[6,
   4121,
   5730,
   9028,
   4,
   994,
   9413,
   994,
   1842,
   11963,
   2603,
   7461,
   1580,
   4663,
   39567,
   9131,
   32,
   2,
   250025],
  [6,
   2672,
   7452,
   156680,
   7153,
   487,
   2458,
   24608,
   1726,
   465,
   14875,
   44090,
   30,
   2,
   250025],
  [6,
   2963,
   72938,
   21646,
   1955,
   56539,
   1677,
   4502,
   4,
   631,
   228616,
   30,
   2,
   250025],
  [6,
   69286,
   123433,
   5364,
   5052,
   20128,
   2308,
   63851,
   40513,
   354,
   36563,
   190011,
   17010,
   186838,
   8857,
   164583,
   2,
   250025],
  [6,
   6607,
   33929,
   131597,
   73130,
   1511,
   19,
   670,
   56256,
   95728,
   10109,
   20556,
   49755,
   2,
   250025],
  [6,
   29787,
   955,
   136664,
   133157,
   15635,
   1903,
   140471,
   99823,
   1189,
   203846,
   2,
   250025],
  [6, 179566, 16405, 213, 175502, 32, 994, 14139, 8689, 2, 250025],
  [57818,
   14439,
   1726,
   4941,
   31874,
   1965,
   109466,
  

## BT

In [2]:
model_path  = "/data/hyxu/lowMT_compute/model/ft-clean-denoising/chpk-24324/model/"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = model.cuda()

In [20]:
mono_path = "/data/hyxu/lowMT_compute/data/public_data/train/mono/bt.nl"
src_lang, tgt_lang = "nl_XX", "zh_CN"
with open(mono_path) as f:
    nl_data = f.readlines()
nl_data = [s.strip("\n") for s in nl_data]
len(nl_data)

500000

In [25]:
src_data = nl_data
num_beams, batch_size = 3, 32
num_batch = len(src_data) // batch_size if len(src_data) % batch_size == 0 else len(src_data) // batch_size +1
predictions = []
model.eval()
with torch.no_grad():
    for i in tqdm(range(num_batch)):
        x = tokenizer(src_data[i*batch_size:(i+1)*batch_size], padding=True, max_length=128, return_tensors="pt",truncation=True)
        for k, v in x.items():
            x[k] = v.to(model.device)
        l = len(x["input_ids"][0])
        
        outputs = model.generate(input_ids=x["input_ids"],
                         attention_mask=x["attention_mask"],
                         num_beams=num_beams,
                         decoder_start_token_id=tokenizer.lang_code_to_id[tgt_lang],
                         max_length= int(l*1.5),)            #mbart
        predictions += tokenizer.batch_decode(outputs.tolist(), skip_special_tokens=True)

  1%|          | 159/15625 [06:00<9:18:30,  2.17s/it] 

In [1]:
predictions[:100]

NameError: name 'predictions' is not defined

In [4]:
# model
predictions = generate(model, tokenizer, src_lang, tgt_lang, nl_data, 32, 4, 128)
len(predictions)

  0%|          | 0/15625 [00:00<?, ?it/s]/data/hyxu/Pruner/src/transformers/src/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (1024) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 26/15625 [04:24<44:02:57, 10.17s/it] 


KeyboardInterrupt: 

In [36]:
batch_size = 100
results_data = results_data[:500000]
num_batch = len(results_data) // batch_size if len(results_data)%batch_size==0 else len(results_data) // batch_size + 1
batch_data = []
for i in tqdm(range(num_batch)):
    x = tokenizer.pad({"input_ids": results_data[i*batch_size: (i+1)*batch_size],
                       "attention_mask": [[1 for _ in range(len(s))] for s in results_data[i*batch_size: (i+1)*batch_size]]}, 
                      padding=True, max_length=128, return_tensors="pt")
    batch_data.append(x)
len(batch_data)

100%|██████████| 5000/5000 [00:05<00:00, 891.19it/s]


5000

In [26]:
model = MBartForConditionalGeneration.from_pretrained(model_path).cuda()
model.eval()

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [38]:
predictions = []
for x in tqdm(batch_data):
    for k, v in x.items():
        x[k] = v.cuda()
    l = len(x["input_ids"][0])
    outputs = model.generate(**x, num_beams=3,
                             decoder_start_token_id=tokenizer.lang_code_to_id[tgt_lang],
                             max_length=int(l * 1.5))
    predictions += tokenizer.batch_decode(outputs, skip_special_tokens=True)
len(predictions), predictions[:10]

100%|██████████| 5000/5000 [3:05:58<00:00,  2.23s/it]  


(500000,
 ['Meneer nummer vier, denk je dat je deze vandaag naar buiten kunt brengen?',
  'En dat dit een belangrijke rol zal spelen in de mondiale humanitaire acties.',
  'Morgen zijn ze nog lekkerder, lieverd.',
  'De toepassing van computer仿真 technologie in de militaire training is een belangrijk onderwerp in defensie.',
  'Een onderzoek naar de effecten van het maken van een nauwkeurige ZnO-kristal.',
  'Het is goed om de directeur te helpen met de grote zaken van het bedrijf.',
  '- Natuurlijk weet je dat.',
  'We moeten de regio uit de conflicten en de oorlogen redden.',
  'De vooruitgang op het gebied van de adaptatie van de bloedtoevoer naar het beschermings mechanisme.',
  'Hoe ziet u de groei van China in de afgelopen jaren?'])

In [41]:
decode_result_data = tokenizer.batch_decode(results_data, skip_special_tokens=True)
list(zip(predictions, decode_result_data))[:100]

[('Meneer nummer vier, denk je dat je deze vandaag naar buiten kunt brengen?',
  '四号先生,你认为你这支花今天能送出去吗?'),
 ('En dat dit een belangrijke rol zal spelen in de mondiale humanitaire acties.',
  '并称这对全球人道行动将有积极的作用。'),
 ('Morgen zijn ze nog lekkerder, lieverd.', '他们明天就会更美味去吧,我亲爱的。'),
 ('De toepassing van computer仿真 technologie in de militaire training is een belangrijk onderwerp in defensie.',
  '计算机仿真技术应用于军事训练是当前国防领域的一项重要课题'),
 ('Een onderzoek naar de effecten van het maken van een nauwkeurige ZnO-kristal.',
  '制备针状ZnO晶须影响因素的研究'),
 ('Het is goed om de directeur te helpen met de grote zaken van het bedrijf.',
  '很好地协助总经理处理公司的重大事务与决议'),
 ('- Natuurlijk weet je dat.', '我不知道你们在说什么?你当然知道'),
 ('We moeten de regio uit de conflicten en de oorlogen redden.',
  '我们必须将该区域从冲突和战争中挽救出来。'),
 ('De vooruitgang op het gebied van de adaptatie van de bloedtoevoer naar het beschermings mechanisme.',
  '心肌缺血预适应的延迟相保护机制的研究进展'),
 ('Hoe ziet u de groei van China in de afgelopen jaren?',
  '中国近年来一直高速增长,您如何看中国的经济发展?'

In [42]:
with open("/data/hyxu/codes/lowMT_compute/data/public_data/train/pair/bt-pair/bt.nl", "w") as f:
    f.write("\n".join(predictions))
with open("/data/hyxu/codes/lowMT_compute/data/public_data/train/pair/bt-pair/bt.zh", "w") as f:
    f.write("\n".join(decode_result_data))